In [1]:
import os
# Check cwd
os.chdir('/home/atom/cvlab/thesis/') 
# os.chdir('/home/member/github/BS-thesis') 
# BS-thesis should be root
print(os.getcwd())

/home/atom/cvlab/thesis


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Dependencies
import glob, itertools
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm_notebook
from functools import partial
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from my_pkg import dataset
from my_pkg import utils

import sys
sys.path.append('cvlab_toolbox')
from cvt.models import SubspaceMethod, MutualSubspaceMethod, KernelMSM, ConstrainedMSM

# Hankel Matrix Model

- Concatenate joints into one vector
- Create the hankel matrix
- make a subspace from the hankel matrix

## Data Preperation

In [4]:
pose_paths = glob.glob('./data/marker/*.csv')
grf_paths = glob.glob('./data/GRF/*.csv')
info_path = './data/info.xlsx'
dataset = dataset.SLJDataset(pose_paths, grf_paths, info_path)

# Concatenate joints into one vector
def transform(sample):
    trunc_pose = sample['trunc_pose']
    F, J, d = trunc_pose.shape
    sample['trunc_pose'] = trunc_pose.reshape(F, J*d)
    return sample    

dataset.transforms = [transform] 
trainset, testset = dataset.train_test_split(test_size = 0.3, stratify=0)

--- Split stats ---
Number of train subjects: 100
Number of test subjects: 43

Subject Ratio: 2.3255813953488373
Sample Ratio: 2.3164556962025316

Labels Ratio ---
Label ['miss', 'healthy', 'structural', 'subjective', 'recovered', 'prone']
Train [0.19398907 0.19398907 0.53005464 0.3442623  0.1147541  0.31420765]
Test [0.15189873 0.24050633 0.43037975 0.29113924 0.15189873 0.29113924]


In [5]:
X_train, y_train = trainset['trunc_pose'], np.asarray(trainset['label'])[:, 0]
X_test, y_test = testset['trunc_pose'], np.asarray(testset['label'])[:, 0]

### Making the Hankel matrix

In [ ]:
def hankel(mat, c):
    C, R = mat.shape
    ret = np.zeros((R*c, C))
    for i in range(c):
        for j in range(C):
            ret[i*R:(i+1)*R, j]= mat[j]
    return ret   

Hs = []
for x in tqdm_notebook(X_train):
    H = hankel(x, 20)
    U,S,V = np.linalg.svd(H)
    Hs.append(U[:5])
    break

KeyboardInterrupt: 